# Setup Colab Env

## Access ARQMath on Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from os import listdir
from os.path import join
dirfiles = listdir("/content/drive/MyDrive/ARQMathAgg/dataset/")
print(dirfiles)

['PostLinks.V1.3.xml', 'Comments.V1.3.xml', 'Votes.V1.3.xml', 'Posts.V1.3.xml', 'README_DATA.md', 'Tags.V1.3.xml', 'Posts.V1.3.zip', 'Users.V1.3.xml', 'Badges.V1.3.xml', 'Old versions', 'Thread Files (supplementary)']


## Clone git repos

In [ ]:
!git clone https://github.com/past12am/MathIR

fatal: destination path 'MathIR' already exists and is not an empty directory.


### Setup ColBERT

In [ ]:
try: # When on google Colab, let's install all dependencies with pip.
    import google.colab
    !pip install -U pip
    !pip install -e ColBERT/['faiss-gpu','torch']
except Exception:
  import sys; sys.path.insert(0, 'ColBERT/')
  try:
    from colbert import Indexer, Searcher
  except Exception:
    print("If you're running outside Colab, please make sure you install ColBERT in conda following the instructions in our README. You can also install (as above) with pip but it may install slower or less stable faiss or torch dependencies. Conda is recommended.")
    assert False

# Index Dataset with ColBERT

In [ ]:
try: # When on google Colab, let's install all dependencies with pip.
    import google.colab
    !pip install -U pip
    !pip install -e ColBERT/['faiss-gpu','torch']
except Exception:
  import sys; sys.path.insert(0, 'ColBERT/')
  try:
    from colbert import Indexer, Searcher
  except Exception:
    print("If you're running outside Colab, please make sure you install ColBERT in conda following the instructions in our README. You can also install (as above) with pip but it may install slower or less stable faiss or torch dependencies. Conda is recommended.")
    assert False

In [ ]:
import os

dataroot = '/content/drive/MyDrive/ARQMathAgg/dataset/'
collection = os.path.join(dataroot, 'collection.tsv')


nbits = 4   # encode each dimension with 2 bits
doc_maxlen = 500   # truncate passages at 300 tokens

checkpoint = 'colbert-ir/colbertv2.0'
index_name = f'arqmath.full.{nbits}bits'

with Run().context(RunConfig(nranks=1, experiment="ColBERTTestARQMath")):

    config = ColBERTConfig(
        nbits=2,
        doc_maxlen=doc_maxlen,
        kmeans_niters=10
    )


    print("Indexing")
    indexer = Indexer(checkpoint=checkpoint, config=config)
    indexer.index(name=index_name, collection=collection, overwrite=True)